In [1]:
!pip install transformers torch datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
#Importamos dependencias
import numpy as np
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import evaluate


In [3]:
#Utilizaremos el dataset imdb (clasificación de sentimientos en películas).
dataset=load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [11]:
#Tokenizamos los datos
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [12]:
#configuramos el modelo Bert
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# Eliminamos la columna "text" para evitar problemas en Trainer
small_train_dataset = small_train_dataset.remove_columns(["text"])
small_eval_dataset = small_eval_dataset.remove_columns(["text"])

# Renombramos "label" a "labels" para evitar problemas con Trainer
small_train_dataset = small_train_dataset.rename_column("label", "labels")
small_eval_dataset = small_eval_dataset.rename_column("label", "labels")
#Convertimos el dataset a formato compatible con PyTorch
small_train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
small_eval_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [14]:
# Configuramos los argumentos del entrenamiento
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8
)

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Agregar prints para depurar
    print(f"Logits: {logits if logits is not None else 'None'}")
    print(f"Labels: {labels if labels is not None else 'None'}")

    if labels is None or logits is None:
        return {}

    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#Configuramos el entrenador de Hugging Face
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.334354,0.876000
2,No log,0.372434,0.889000


Logits: [[-1.415734   2.1279275]
 [-0.7442919  1.1915917]
 [ 1.7635791 -2.2069182]
 ...
 [ 1.4190056 -1.8359468]
 [-1.8283937  2.3690019]
 [-1.899599   2.4497433]]
Labels: [1 1 0 1 0 1 1 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 1 0 1 0 1 1 1 0 0 1 0 0 1 0
 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0 1
 0 0 0 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1
 1 1 1 0 1 1 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 1 1 1 0 0 1 1 0 1 1 0 1 0 0 0
 1 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 0 0 1 1 1 1 1 1 1 0 0
 1 0 0 1 0 1 0 0 1 1 1 0 1 1 1 1 0 0 0 0 0 1 0 0 0 1 1 1 0 0 1 1 1 0 1 0 0
 0 0 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 1 1 0 0 1 0 1 0 1 1 0 0 0 1 1 1 1 1 1
 1 1 0 1 0 0 1 1 0 1 0 1 1 0 1 1 1 0 1 1 1 1 0 0 1 0 1 0 1 1 0 1 0 0 1 0 1
 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 1
 0 1 0 0 0 1 1 1 0 1 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 1 1 1 0 1
 1 0 1 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 0 1 1 1 1 1 0 1 0 1 0 1
 1 

TrainOutput(global_step=250, training_loss=0.34276992797851563, metrics={'train_runtime': 57.1039, 'train_samples_per_second': 35.024, 'train_steps_per_second': 4.378, 'total_flos': 526222110720000.0, 'train_loss': 0.34276992797851563, 'epoch': 2.0})

In [17]:
evaluation_results = trainer.evaluate()
# 🔎 Mostramos los resultados de la evaluación
print("\n📊 📌 Resultados Finales del Modelo:")
print(f"- 📉 Pérdida en evaluación (eval_loss): {evaluation_results['eval_loss']:.4f}")
print(f"- 🎯 Precisión en evaluación (eval_accuracy): {evaluation_results['eval_accuracy'] * 100:.2f}%")
print(f"- ⏳ Tiempo total de evaluación (eval_runtime): {evaluation_results['eval_runtime']:.2f} segundos")
print(f"- ⚡ Muestras por segundo (eval_samples_per_second): {evaluation_results['eval_samples_per_second']:.2f}")
print(f"- 🔄 Pasos por segundo (eval_steps_per_second): {evaluation_results['eval_steps_per_second']:.2f}")
print(f"- 🏁 Época alcanzada: {evaluation_results['epoch']}")



Logits: [[-2.2700126   2.7868223 ]
 [-0.67870945  1.1484199 ]
 [ 2.1702986  -2.6150703 ]
 ...
 [ 2.1365993  -2.582158  ]
 [-2.4209337   2.9395497 ]
 [-2.413781    2.9776003 ]]
Labels: [1 1 0 1 0 1 1 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 1 0 1 0 1 1 1 0 0 1 0 0 1 0
 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 0 0 0 0 1 0 0 1 1 0 0 0 0 1
 0 0 0 1 0 1 0 1 0 1 1 1 1 0 1 1 0 1 0 1 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 1
 1 1 1 0 1 1 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 0 1 1 1 0 0 1 1 0 1 1 0 1 0 0 0
 1 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 0 0 1 1 1 1 1 1 1 0 0
 1 0 0 1 0 1 0 0 1 1 1 0 1 1 1 1 0 0 0 0 0 1 0 0 0 1 1 1 0 0 1 1 1 0 1 0 0
 0 0 1 1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 1 1 0 0 1 0 1 0 1 1 0 0 0 1 1 1 1 1 1
 1 1 0 1 0 0 1 1 0 1 0 1 1 0 1 1 1 0 1 1 1 1 0 0 1 0 1 0 1 1 0 1 0 0 1 0 1
 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0 0 0 1 0 1 1
 0 1 0 0 0 1 1 1 0 1 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 1 1 1 0 1
 1 0 1 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 0 1 0 1 1 1 1 1 0 1